In [110]:
# 1. Katie Corcoran

In [111]:
# 2.
import warnings
warnings.filterwarnings('ignore')

import pymssql
import pandas as pd

conn = pymssql.connect(
     host='HOST', # Server name goes in quotes.
     user='USER', # Username goes in quotes.
     password='PASSWORD', # Password goes in quotes
     database='overland') # Database to use goes in quotes

query = """
WITH HouseholdData AS
(SELECT 
    HHID,
    COUNT(CASE WHEN Guest.SeasonPass = 'y' THEN 1 END) AS SPCount,
    COUNT(CASE WHEN Guest.Age < 18 THEN 1 END) AS U18Count,
    COUNT(*) AS Members
FROM Guest
GROUP BY HHID),
VisitData AS
(SELECT 
    HHID,
    COUNT(Visit.VisID) AS VisitCt
FROM Guest
JOIN Visit ON Visit.CustID = Guest.GuestID
GROUP BY HHID)

SELECT DISTINCT
    Household.HHID,
    Household.Region,
    Household.HHI AS Income,
    Guest.LName,
    HouseholdData.Members,
    HouseholdData.SPCount,
    HouseholdData.U18Count,
    VisitData.VisitCt
FROM Household
JOIN HouseholdData ON HouseholdData.HHID = Household.HHID
JOIN Guest ON Guest.HHID = Household.HHID
JOIN VisitData ON VisitData.HHID = Household.HHID
ORDER BY HHID
"""
df = pd.read_sql(query, conn, index_col='HHID')
print(df)

          Region  Income        LName  Members  SPCount  U18Count  VisitCt
HHID                                                                      
1     Western PA  153000       Willis        2        0         1       12
2     Western PA  129000      Simpson        3        0         2        6
3          Other   68000  Sonic-Smith        1        0         0        6
4     Central PA   98000      Elliott        1        0         0        3
5     Central PA   52000        Owens        1        1         0        6
...          ...     ...          ...      ...      ...       ...      ...
7151  Central PA   36000     Mitchell        1        0         0        2
7152  Western PA   85000     Mitchell        1        0         0        6
7153     DC Area   72000        Green        1        0         0        5
7154  Central PA   48000    Schroeder        1        0         0        3
7155  Western PA  111000         Carr        1        0         1        3

[7088 rows x 7 columns]


In [112]:
# 3.
import numpy as np

df.replace({'SPCount': np.nan}, value=0, inplace=True)
df.replace({'U18Count': np.nan}, value=0, inplace=True)
df.replace({'VisitCt': np.nan}, value=0, inplace=True)
print(df)

          Region  Income        LName  Members  SPCount  U18Count  VisitCt
HHID                                                                      
1     Western PA  153000       Willis        2        0         1       12
2     Western PA  129000      Simpson        3        0         2        6
3          Other   68000  Sonic-Smith        1        0         0        6
4     Central PA   98000      Elliott        1        0         0        3
5     Central PA   52000        Owens        1        1         0        6
...          ...     ...          ...      ...      ...       ...      ...
7151  Central PA   36000     Mitchell        1        0         0        2
7152  Western PA   85000     Mitchell        1        0         0        6
7153     DC Area   72000        Green        1        0         0        5
7154  Central PA   48000    Schroeder        1        0         0        3
7155  Western PA  111000         Carr        1        0         1        3

[7088 rows x 7 columns]


In [113]:
# 4.
df['VisPP'] = df['VisitCt'] / df['Members']
print(df[['LName', 'Members', 'VisitCt', 'VisPP']])

            LName  Members  VisitCt  VisPP
HHID                                      
1          Willis        2       12    6.0
2         Simpson        3        6    2.0
3     Sonic-Smith        1        6    6.0
4         Elliott        1        3    3.0
5           Owens        1        6    6.0
...           ...      ...      ...    ...
7151     Mitchell        1        2    2.0
7152     Mitchell        1        6    6.0
7153        Green        1        5    5.0
7154    Schroeder        1        3    3.0
7155         Carr        1        3    3.0

[7088 rows x 4 columns]


In [114]:
# 5.
df['IsLocal'] = np.where((df['Region'] == 'Central PA'), 1, 0)
print(df[['LName', 'Region', 'IsLocal']])

            LName      Region  IsLocal
HHID                                  
1          Willis  Western PA        0
2         Simpson  Western PA        0
3     Sonic-Smith       Other        0
4         Elliott  Central PA        1
5           Owens  Central PA        1
...           ...         ...      ...
7151     Mitchell  Central PA        1
7152     Mitchell  Western PA        0
7153        Green     DC Area        0
7154    Schroeder  Central PA        1
7155         Carr  Western PA        0

[7088 rows x 3 columns]


In [115]:
# 6.
dflocal = df[df['IsLocal'] == 1]
print(dflocal)

          Region  Income      LName  Members  SPCount  U18Count  VisitCt  \
HHID                                                                       
4     Central PA   98000    Elliott        1        0         0        3   
5     Central PA   52000      Owens        1        1         0        6   
6     Central PA  159000  Underwood        2        2         1       18   
7     Central PA  138000       Wise        1        0         1        5   
8     Central PA   78000      Tyler        6        0         4       22   
...          ...     ...        ...      ...      ...       ...      ...   
7143  Central PA   80000     Molina        1        0         0        9   
7145  Central PA  134000      Scott        5        0         4       33   
7149  Central PA  139000     Watson        2        0         0       71   
7151  Central PA   36000   Mitchell        1        0         0        2   
7154  Central PA   48000  Schroeder        1        0         0        3   

          V

In [116]:
# 7.
dflocal.drop(columns=['IsLocal', 'Region'], inplace=True);
print(dflocal)

      Income      LName  Members  SPCount  U18Count  VisitCt      VisPP
HHID                                                                   
4      98000    Elliott        1        0         0        3   3.000000
5      52000      Owens        1        1         0        6   6.000000
6     159000  Underwood        2        2         1       18   9.000000
7     138000       Wise        1        0         1        5   5.000000
8      78000      Tyler        6        0         4       22   3.666667
...      ...        ...      ...      ...       ...      ...        ...
7143   80000     Molina        1        0         0        9   9.000000
7145  134000      Scott        5        0         4       33   6.600000
7149  139000     Watson        2        0         0       71  35.500000
7151   36000   Mitchell        1        0         0        2   2.000000
7154   48000  Schroeder        1        0         0        3   3.000000

[2859 rows x 7 columns]


In [117]:
# 8.
dflocal.dropna()
conditions = [
    dflocal['Income'] < 50000,
    dflocal['Income'] < 100000,
    dflocal['Income'] < 150000,
    dflocal['Income'] >= 150000
]
values = [
    'Under $50K',
    '$50-100K',
    '$100-150K',
    'Over $150K'
]
dflocal['IncomeCat'] = np.select(conditions, values)
print(dflocal[['LName', 'Income', 'IncomeCat', 'VisPP']])

          LName  Income   IncomeCat      VisPP
HHID                                          
4       Elliott   98000    $50-100K   3.000000
5         Owens   52000    $50-100K   6.000000
6     Underwood  159000  Over $150K   9.000000
7          Wise  138000   $100-150K   5.000000
8         Tyler   78000    $50-100K   3.666667
...         ...     ...         ...        ...
7143     Molina   80000    $50-100K   9.000000
7145      Scott  134000   $100-150K   6.600000
7149     Watson  139000   $100-150K  35.500000
7151   Mitchell   36000  Under $50K   2.000000
7154  Schroeder   48000  Under $50K   3.000000

[2859 rows x 4 columns]


In [142]:
# 9.
print(dflocal[['IncomeCat', 'VisPP']].groupby('IncomeCat').describe())
print('Yes, there is likely a relationship between income and number of visits per person. Households making over $150K visit the most, with $100-150K, $50-100K, and under $50K following behind from most to least, respectively.')



             VisPP                                                         \
             count      mean       std       min       25%       50%  75%   
IncomeCat                                                                   
$100-150K   1117.0  6.057126  3.830559  0.666667  3.750000  5.333333  7.5   
$50-100K    1239.0  4.849318  2.772813  0.500000  3.000000  4.000000  6.0   
Over $150K   445.0  7.549813  4.357555  0.600000  4.666667  7.000000  9.0   
Under $50K    58.0  3.958621  1.694473  1.000000  3.000000  4.000000  5.0   

                  
             max  
IncomeCat         
$100-150K   35.5  
$50-100K    31.0  
Over $150K  31.0  
Under $50K   8.0  
Yes, there is likely a relationship between income and number of visits per person. Households making over $150K visit the most, with $100-150K, $50-100K, and under $50K following behind from most to least, respectively.


In [146]:
# 10.
print(dflocal[['Income', 'Members', 'SPCount', 'U18Count', 'VisPP']].corr())
print('Income has the strongest relationship with visits per person for local guests because the correlation coefficient between VisPP and Income has the highest absolute value.')
print('The number of members in the household has the weakest relationship with visits per person, because the correlation coefficient between VisPP and Members has the lowest absolute value.')



            Income   Members   SPCount  U18Count     VisPP
Income    1.000000  0.350453  0.396241  0.290968  0.283979
Members   0.350453  1.000000  0.194179  0.903288 -0.119488
SPCount   0.396241  0.194179  1.000000  0.169623  0.231840
U18Count  0.290968  0.903288  0.169623  1.000000 -0.119970
VisPP     0.283979 -0.119488  0.231840 -0.119970  1.000000
Income has the strongest relationship with visits per person for local guests because the correlation coefficient between VisPP and Income has the highest absolute value.
The number of members in the household has the weakest relationship with visits per person, because the correlation coefficient between VisPP and Members has the lowest absolute value.
